## Machine Learning: Initialize Trained Network

In [1]:
#PyTorchの初期化
import torch
import torchvision

#GPIO
import Jetson.GPIO as GPIO

#ランダム関数
import random

model = torchvision.models.alexnet(pretrained=False)
#種類を増やした場合、変更するところ
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 3)

#モデルの読み込み
model.load_state_dict(torch.load('best_model.pth'))

#カメラのフォーマットを学習済みモデルのフォーマットに合わせ変換
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

#カメラ画像の取得
from jetcam.csi_camera import CSICamera
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224)
image = camera.read()
image_widget = ipywidgets.Image(format='jpeg')
image_widget.value = bgr8_to_jpeg(image)

#カメラ画像の更新
camera.running = True

def update_image(change):
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)
    
camera.observe(update_image, names='value')

#GPU
device = torch.device('cuda')
model = model.to(device)


## Machine Learning: Deduction using the trained network

In [2]:
#推論
import torch.nn.functional as F
import time
import sys
a=0
one_blocked=0.0
two_blocked=0.0
three_blocked=0.0
def update(change):
    global blocked_slider, robot,a
    global one_blocked, two_blocked, three_blocked
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    one_blocked = float(y.flatten()[0])
    two_blocked = float(y.flatten()[1])
    three_blocked = float(y.flatten()[2])
    if(one_blocked > two_blocked and one_blocked > three_blocked):
        a = 0
    elif(two_blocked > one_blocked and two_blocked > three_blocked):
        a = 1
    elif(three_blocked > one_blocked and three_blocked > one_blocked):
        a = 2
    #0　グー rock 
    #1　パー paper
    #2　チョキ scissors   
    
    time.sleep(0.001)
        
update({'new': image})  # we call the function once to intialize


## Gameplay: imports PWM and other libaries

In [3]:
### PWM imports
# PWM is used for ratation of a servo, 
# which is used to indicate the decision 
# (rock, paper or scissor) of the Jetson Nano.
###
import Fabo_PCA9685
import pkg_resources
SMBUS='smbus'
for dist in pkg_resources.working_set:
    #print(dist.project_name, dist.version)
    if dist.project_name == 'smbus':
        break
    if dist.project_name == 'smbus2':
        SMBUS='smbus2'
        break
if SMBUS == 'smbus':
    import smbus
elif SMBUS == 'smbus2':
    import smbus2 as smbus

# init
BUSNUM=1
SERVO_HZ=50
INITIAL_VALUE=300
bus = smbus.SMBus(BUSNUM)
PCA9685 = Fabo_PCA9685.PCA9685(bus,INITIAL_VALUE,address=0x60)
PCA9685.set_hz(SERVO_HZ)

# set initial value, to a value that is neither R, P nor S
PCA9685.set_channel_value(3, 220) # PWM_CHANNEL, PWM_VALUE


### import randoms
import math

### import colors
from colorama import Fore, Back, Style


## Gameplay: imports GPIO library; actual gameplay process

In [ ]:
### GPIO imports
# GPIO is mainly used for controlling lights
# and recieving signals from a button
#
# > Note: GPIO needs to be set every 
#   time when the code bloack is run.
###
import Jetson.GPIO as GPIO

# initialize
GPIO.setwarnings(False)
GPIO.setmode( GPIO.BCM )

# pins initialize
RED_OR_GREEN_CHOOSER_PIN = 13
RED = GPIO.LOW#False
GREEN = GPIO.HIGH#True
YELLOW_PIN = 16
YELLOW_ON = GPIO.LOW
YELLOW_OFF = GPIO.HIGH
BUTTON_PIN = 5

GPIO.setup( [RED_OR_GREEN_CHOOSER_PIN, YELLOW_PIN], GPIO.OUT )
GPIO.setup( BUTTON_PIN, GPIO.IN )

### Functions
def input_opponent_value_from_GPIO():
    return math.floor(random.random()*3)

def name_of_value(val):
    if val == 0:
        return "Rock    ";
    if val == 1:
        return "Paper   ";
    if val == 2:
        return "Scissor ";

### Dances when result is out (TODO)
def tie_game():
    GPIO.output( YELLOW_PIN, YELLOW_ON )
    GPIO.output( RED_OR_GREEN_CHOOSER_PIN, RED )
    PCA9685.set_channel_value(3,537)
    time.sleep( 0.5 )
    GPIO.output( RED_OR_GREEN_CHOOSER_PIN, GREEN )
    PCA9685.set_channel_value(3,105)
    time.sleep( 0.5 )  

def won_game():
    # pin reset
    GPIO.output( YELLOW_PIN, YELLOW_ON )
    PCA9685.set_channel_value(3,537)
    # pin pattern
    GPIO.output( RED_OR_GREEN_CHOOSER_PIN, GREEN )
    PCA9685.set_channel_value(3,105)
    time.sleep( 0.3 )
    GPIO.output( YELLOW_PIN, YELLOW_OFF )
    PCA9685.set_channel_value(3,321)
    time.sleep( 0.3 )
    GPIO.output( RED_OR_GREEN_CHOOSER_PIN, RED )
    PCA9685.set_channel_value(3,537)
    time.sleep( 0.3 )
    GPIO.output( YELLOW_PIN, YELLOW_ON )
    PCA9685.set_channel_value(3,105)
    time.sleep( 0.3 ) 

def lost_game():
    for i in range(105, 537, 10):
        PCA9685.set_channel_value(3,i)
        time.sleep( 0.1 ) 
    for i in range(0, 537 - 105, 10):
        PCA9685.set_channel_value(3,537 - i)
        time.sleep( 0.1 ) 
    
### MAIN

# activate hand gesture recognition
display(image_widget)
camera.observe(update, names='value')

# main process
game_count = won_count = 0
try:
    while True: # forever loop
        # wait for signal
        sys.stdout.write("\n\rPush for new game.")  
        GPIO.wait_for_edge(BUTTON_PIN, GPIO.RISING)
        
        # reset light and rotation
        game_count = game_count+1
        GPIO.output( RED_OR_GREEN_CHOOSER_PIN, RED )
        GPIO.output( YELLOW_PIN, YELLOW_OFF )
        PCA9685.set_channel_value(3,220) 
        sys.stdout.flush()
        sys.stdout.write("\rGame %2s: 3" %(i))
        time.sleep( 1 )    
        
        # Rock-
        GPIO.output( RED_OR_GREEN_CHOOSER_PIN, RED )
        GPIO.output( YELLOW_PIN, YELLOW_OFF )
        sys.stdout.flush()        
        sys.stdout.write("\rGame %2s: 2" %(i))
        time.sleep( 1 )    
        
        # Paper-
        GPIO.output( RED_OR_GREEN_CHOOSER_PIN, RED )
        GPIO.output( YELLOW_PIN, YELLOW_ON )
        sys.stdout.flush()
        sys.stdout.write("\rGame %2s: 1" %(i))
        time.sleep( 1 )
              
        # Scissors (GO!)
        GPIO.output( RED_OR_GREEN_CHOOSER_PIN, GREEN )
        GPIO.output( YELLOW_PIN, YELLOW_OFF )
        sys.stdout.flush()
        sys.stdout.write("\rGame %2s: GO!" %(i))
        
        rint = random.randint(0,2)
        PCA9685.set_channel_value(3,105+rint*216)
              
        # Wait a little and detect hand gesture
        time.sleep( 2 )
        sint = a; #input_opponent_value_from_GPIO();#input_opponent_value_from_camera() - 1;
        sys.stdout.write("\rGame %2s  " %(i))
        sys.stdout.write("Opponent: %s You: %s " % (name_of_value(rint), name_of_value(sint)))  
        sys.stdout.write("\tR:%f, P:%f, S:%f\t" % (one_blocked, two_blocked, three_blocked) )
        
        # win or lose
        diff = 0
        diff = (rint - sint) % 3;
        if diff == 0:
            sys.stdout.write(Fore.YELLOW+"Draw")
            #tie_game();
        elif diff == 1:
            sys.stdout.write(Fore.RED+"You lose")
            #won_game();
        elif diff == 2:
            sys.stdout.write(Fore.GREEN + "You win!!")
            won_count = won_count + 1
            #lost_game();
        sys.stdout.write(Style.RESET_ALL)
        
except KeyboardInterrupt:
    GPIO.cleanup()
    sys.stdout.write(Style.RESET_ALL + "\n")
    # statistics at the end
    if game_count != 0:
        print("Toatal score: %s/%s\n" % (won_count, game_count) )
    sys.exit(0)
   

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…


Game  1  Opponent: Rock    You: Rock    	R:0.539096, P:0.037684, S:0.423220	Draw
Game  2  Opponent: Scissor  You: Scissor  	R:0.325202, P:0.062623, S:0.612175	Draw
Game  3  Opponent: Paper   You: Paper   	R:0.259968, P:0.701870, S:0.038163	Draw
Game  4  Opponent: Paper   You: Rock    	R:0.604052, P:0.075205, S:0.320743	You lose
Game  5  Opponent: Paper   You: Scissor  	R:0.250346, P:0.022758, S:0.726897	You win!!
Game  6  Opponent: Rock    You: Rock    	R:0.500019, P:0.206409, S:0.293571	Draw
Game  7  Opponent: Scissor  You: Paper   	R:0.204045, P:0.720298, S:0.075658	You lose
Push for new game.